#### TFT API 에 대해서 설명되어있는 사이트 : https://velog.io/@twa02189/TFT-API-%EB%8D%B0%EC%9D%B4%ED%84%B0-%EC%84%A4%EB%AA%85

In [4]:
import pandas as pd
import requests
import pprint
import json
import time
from pandas import json_normalize

pd.set_option('display.max_colwidth', None) ## 셀 최대 너비 제한 해제

#### 기본 키 저장 + baser URL 저장 + request header 저장

In [5]:
# api_key 저장
api_key = 'RGAPI-b2bd2659-5fb4-4be0-821a-e3af369cc138' ## 만료되면 갱신해줘야함 거의 하루마다

# request header 저장
request_header = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36",
    "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
    "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
    "Origin": "https://developer.riotgames.com",
    "X-Riot-Token": api_key 
    }

# base_url 저장
base_url = "https://kr.api.riotgames.com/tft/" ## 이 뒤에다가 + 로 그때그떄 필요한 URL 을 더해줄 것

#### 챌린저, 마스터, 그랜드 마스터 구간 유저들의 닉네임 추출 

##### LEAGUE/V1 API 에서 랭크 구간별 유저 정보 REQUEST

In [6]:
# challenger

challenger_url = base_url + 'league/v1/challenger'

challenger = requests.get(challenger_url, headers = request_header).json()

# grand master

grandmaster_url = base_url + 'league/v1/grandmaster'

grandmaster = requests.get(grandmaster_url, headers = request_header).json()

# master

master_url = base_url + 'league/v1/master'

master = requests.get(master_url, headers = request_header).json()

In [7]:
# 가져온 마스터 데이터 프레임 조회 (마스터 랭크에 속한 유저 조회)
pd.DataFrame(challenger).head(3)

# 가져온 그랜드마스터 데이터 프레임 조회 (그랜드마스터 랭크에 속한 유저 조회)
pd.DataFrame(grandmaster).head(3)

# 가져온 마스터 데이터 프레임 조회 (마스터 랭크에 속한 유저 조회)
pd.DataFrame(master).head(3)

,tier,leagueId,queue,name,entries
0,MASTER,c9353fab-a304-3562-bdf5-e7375f43e1cd,RANKED_TFT,Karthus's Executioners,"{'summonerId': 'kYM3veK_wr19-BR0G8nCrxJUpw6XydujUBtsLC5Z_Ew-BY363sMVqxUz-Q', 'summonerName': '아카드1', 'leaguePoints': 63, 'rank': 'I', 'wins': 82, 'losses': 52, 'veteran': False, 'inactive': False, 'freshBlood': True, 'hotStreak': False}"
1,MASTER,c9353fab-a304-3562-bdf5-e7375f43e1cd,RANKED_TFT,Karthus's Executioners,"{'summonerId': 'JMem_L6OA9U5A74P22U3YgpSYCtMdJmGEKq3_8XM-opTM2U', 'summonerName': '외과울산 쉔', 'leaguePoints': 26, 'rank': 'I', 'wins': 86, 'losses': 62, 'veteran': False, 'inactive': False, 'freshBlood': True, 'hotStreak': True}"
2,MASTER,c9353fab-a304-3562-bdf5-e7375f43e1cd,RANKED_TFT,Karthus's Executioners,"{'summonerId': '3mCETHngT_vIRek_Wt00PVvY4DdOMFdasWS0MIfwETSzh_ae8ZaPR4kHSw', 'summonerName': 'Acro OHMYEYE', 'leaguePoints': 0, 'rank': 'I', 'wins': 88, 'losses': 73, 'veteran': False, 'inactive': False, 'freshBlood': True, 'hotStreak': False}"


##### 추출한 랭크 구간별 유저 정보 데이터 프레임화

In [8]:
# challenger_df 에 저장
challenger_df = pd.DataFrame(challenger)

# grandmaster_df 에 저장
grandmaster_df = pd.DataFrame(grandmaster)

# master_df 에 저장
master_df = pd.DataFrame(master)

##### json Normalize, 필요없는 컬럼 drop 하는 함수 생성

In [9]:
def json_normalize_and_dropna(df):
    
    # entries 컬럼 확장  -  json 형식의 entries  컬럼 안에 중요 정보가 대부분 담겨있음
    entries_normalized = json_normalize(df['entries'])
    
    # 확장된 데이터프레임 병합
    df = pd.concat([df.drop(['entries'], axis=1), entries_normalized], axis=1)
    
    # 필요없는 컬럼 drop('queue', 'name')
    df = df.drop(['leagueId', 'queue', 'name', 'rank', 'veteran', 'inactive', 'freshBlood', 'hotStreak'], axis=1)
    
    return df

##### 챌린저, 그랜드 마스터, 마스터 유저 정보 데이터 프레임에 함수 적용

In [10]:
# challenger df 결측치 제거 및 json normalize
# challenger_df = json_normalize_and_dropna(challenger_df) ## 현재 챌린저가 없어서 에러 발생하므로 주석 처리함

# grandmaster df 결측치 제거 및 json normalize
grandmaster_df = json_normalize_and_dropna(grandmaster_df)

# grandmaster df 결측치 제거 및 json normalize
master_df = json_normalize_and_dropna(master_df)

In [11]:
grandmaster_df.head(3)

,tier,summonerId,summonerName,leaguePoints,wins,losses
0,GRANDMASTER,9sxIws6CI6fmK2sw2cAo_Xe8E82DNcR9XtHkWKC6W14SK2w,틈 빈,265,51,10
1,GRANDMASTER,BhJKOIvlHJRa_Ra0uNNDlrqN46i6snagNkH_p8VZR28GJA,Teukhan,327,84,55
2,GRANDMASTER,80HtOC_fs-G1sNBcJFSlZXfNcYfz6he1pHYfBpJHkBs_2TM,COPY O,449,81,38


##### 소환사 이름 데이터 프레임을 넣으면 닉네임에 대응하는 puuid 를 붙여서 반환하는 함수 생성

##### 먼저 닉네임을 넣으면 대응하는 puuid 를 반환하는 함수 생성

In [12]:
def get_puuid(summonerName):
    # 닉네임을 넣어줄 url 저장
    summonerName_url = base_url + "summoner/v1/summoners/by-name/" + summonerName

    # 닉네임을 넣었을 때, 반환값 저장
    have_puuid_table = requests.get(summonerName_url, headers = request_header).json()

    # puuid 추출
    return have_puuid_table['puuid']

In [13]:
# 함수 작동 테스트
get_puuid('COPY O')

't_Wf-ad5Drr5FzTmxmkRnsEtBNUf-7PRoTll2k8kxN23sYTKJs2VOrOjeQ418PF31x2QQ2X02RrAtw'

##### get_puuid 함수를 이용해서 대응하는 puuid 를 새로운 컬럼으로 추가하여 반환하는 함수 생성

In [14]:
def get_puuid_column(df):
    # 새로운 컬럼을 추가할 리스트 생성
    puuid_list = []

    # 데이터프레임에서 소환사 이름 컬럼 찾기
    summonerName_column = None
    for column in df.columns:
        if 'summonerName' in column:
            summonerName_column = column
            break
    
    if summonerName_column is None:
        print("No column containing summoner names found in the dataframe.")
        return df

    # 각 소환사 이름에 대해 puuid 가져오기
    for summonerName in df[summonerName_column]:
        try:
            # 소환사 이름으로 puuid 가져오기
            puuid = get_puuid(summonerName)
            puuid_list.append(puuid)
        except Exception as e:
            print(f"Error getting PUUID for summoner '{summonerName}': {str(e)}")
            puuid_list.append(None)  # 에러가 발생하면 None을 추가

    # 새로운 컬럼을 데이터프레임에 추가
    df['puuid'] = puuid_list

    return df

##### get_puuid_column(df) 함수 챌,그마,마 테이블에 적용

In [15]:
# challanger
# get_puuid_column(challanger_df) # 차후에 챌린저 생기면 주석 제거 필요

# grandmaster
get_puuid_column(grandmaster_df)

# master
get_puuid_column(master_df)

Error getting PUUID for summoner '': 'puuid'
Error getting PUUID for summoner '': 'puuid'
Error getting PUUID for summoner '': 'puuid'
Error getting PUUID for summoner '': 'puuid'


,tier,summonerId,summonerName,leaguePoints,wins,losses,puuid
0,MASTER,kYM3veK_wr19-BR0G8nCrxJUpw6XydujUBtsLC5Z_Ew-BY363sMVqxUz-Q,아카드1,63,82,52,OfzVbDs0P0ypb4kaakCEDMiO9u8K5KMccwj65AU_rcvTlKVGOdA_neQuoJWq1Xljp8wujSJ0_EYcxA
1,MASTER,JMem_L6OA9U5A74P22U3YgpSYCtMdJmGEKq3_8XM-opTM2U,외과울산 쉔,26,86,62,00kdRgJGjM4y9d5syi1Ntc4i8DgPSF1_YDNu-iOUB5CuIV8ex_rzcWMjyaRihfOroeQoRHHptDuAEw
2,MASTER,3mCETHngT_vIRek_Wt00PVvY4DdOMFdasWS0MIfwETSzh_ae8ZaPR4kHSw,Acro OHMYEYE,0,88,73,bAUTBjhuA00n1167MUmonjyMgetU3Dv1-5IGNUqWwWt-59C4f_W2hhtPLqZ5KQTNdoQCjExuuGszZg
3,MASTER,Me6VA4XyXI5CW9Wym_6TE7JxQzPXLIs3c680VfWU-rDhSQ5JqWL49A8z-Q,,185,83,34,None
4,MASTER,gtmxIzxrWvshGAbgRQ0H8xiM8_Db1TENqogrCwTDpi3WDalRv3naRyULqg,krobab,102,113,78,LZ6lv1UG0xUMVNyzu-R4xTNkHietPz1rLIdevEkkzaTzQc15A3FZvJR1BnDIVnDvXJHcRay84sZCQw
5,MASTER,xszNgAg-6vRPAVyxqeu3L9oAeiFW8OnunPj6wbAbwAq_wVw,존잘인천피스트,214,88,68,7ITuO8wSX3-KnWipQVyAnnx7y75RPmOOU2TVy42-fSya7UXdwHfioyIG7jn6owP-tkUzrPfOU7Q4RQ
6,MASTER,ULU0KTI8zQ0Ds8pOo8zPrqxeLL-nWf10EqpnK95zWIlvP3XPgj3zNgwBbQ,duddhzns,0,82,62,iYqxpOOA5Vz_4QgUSkv4qyZbyE8vLwB5WpHnp9HhMly6ofuIL00hqpgXr9Vgcqbdc9arzvxHK_-XGQ
7,MASTER,-z8d2-ODI2-7nVCYuX4_SLOdugvvgAJghBN9jgEGDtJRezwkruPqaHtJpQ,그저게임가지고왜,6,52,26,KuW8RG4H1TlI9aDlPgNF4Fh_sgeqwLYyFq060o_eiJSQj30TRVpmM1mzvbI2jLmGnVseIXen7kGwHA
8,MASTER,ejplkuJkm0edS0SuGLEZH5iXGdTxTQYxeVhyPsvE0V6tlQ,탑 렝가,27,108,88,BLKLr0C6Px7qhf5vNe5jyzYIK2h_QRO2T69teeCtMMU8rGEAV3ZK2hpKlZemnjl1Q6Ti5XnY5G1lRw
9,MASTER,Cx1pSs3OJmBnJyMjaC8-T5ZrCQjES0pjIOV5QZxTJpQ5ulg,ema50cm,8,87,69,XwRIqTXBnBYgAU3Jct9FIDtrIOy01HBn4PUM6BecUex8D2MRFJlhDu2fcAvohMYPQrRGJK65ccfeCQ


In [16]:
# 확인
grandmaster_df.head()

,tier,summonerId,summonerName,leaguePoints,wins,losses,puuid
0,GRANDMASTER,9sxIws6CI6fmK2sw2cAo_Xe8E82DNcR9XtHkWKC6W14SK2w,틈 빈,265,51,10,x5Aluxt-SEq_IUikKvH83sk0EkLXS7CP41bf66M_mBNdsiIUKYdavmHUPDKeWUBL6Ki5WwihBw2kQg
1,GRANDMASTER,BhJKOIvlHJRa_Ra0uNNDlrqN46i6snagNkH_p8VZR28GJA,Teukhan,327,84,55,Qd68F2UMKe7FMb-qpbUs3njG6k_4ZLwr2FRG0zHruSDr9XTtZOZ1cdAXyLd_kGAt0_IheL5QMQLdvQ
2,GRANDMASTER,80HtOC_fs-G1sNBcJFSlZXfNcYfz6he1pHYfBpJHkBs_2TM,COPY O,449,81,38,t_Wf-ad5Drr5FzTmxmkRnsEtBNUf-7PRoTll2k8kxN23sYTKJs2VOrOjeQ418PF31x2QQ2X02RrAtw
3,GRANDMASTER,pNUDUaztjVI1G7Jzoh4aVDMGr_8nDd0sMVeuWOvzwNMC6vB5,MessiOfTFT,394,86,53,4_-cZaXUQM6l7UQGnB69F9007HRO3dyTdKP-7_u0utPCg6YeZCBKMOZvoLFUdNyDa78Y7RBVqBXcGQ
4,GRANDMASTER,VVpwkqbuN3VVwG378diAEHea6jFrq1ADYBQAEstZizthf7k,이렇게 좋은 날,321,94,51,rx1nmyxEddDyQujL2nvjfiHV_LDcBEo-zyf16l3BFnXQExOMoB6tNss-WGlc_GRgcac2tg1GhDgBhg


##### 만들 테이블들(챌린저, 그마, 마스터) 합쳐서 하나의 테이블로 만들기

In [17]:
# 챌린저 티어 유저 생기면 아래에 있는 쿼리 주석 제거하고 사용
#combined_df = pd.concat([challenger_df, grandmaster_df, master_df], ignore_index=True)

combined_df = pd.concat([grandmaster_df, master_df], ignore_index=True)

combined_df

,tier,summonerId,summonerName,leaguePoints,wins,losses,puuid
0,GRANDMASTER,9sxIws6CI6fmK2sw2cAo_Xe8E82DNcR9XtHkWKC6W14SK2w,틈 빈,265,51,10,x5Aluxt-SEq_IUikKvH83sk0EkLXS7CP41bf66M_mBNdsiIUKYdavmHUPDKeWUBL6Ki5WwihBw2kQg
1,GRANDMASTER,BhJKOIvlHJRa_Ra0uNNDlrqN46i6snagNkH_p8VZR28GJA,Teukhan,327,84,55,Qd68F2UMKe7FMb-qpbUs3njG6k_4ZLwr2FRG0zHruSDr9XTtZOZ1cdAXyLd_kGAt0_IheL5QMQLdvQ
2,GRANDMASTER,80HtOC_fs-G1sNBcJFSlZXfNcYfz6he1pHYfBpJHkBs_2TM,COPY O,449,81,38,t_Wf-ad5Drr5FzTmxmkRnsEtBNUf-7PRoTll2k8kxN23sYTKJs2VOrOjeQ418PF31x2QQ2X02RrAtw
3,GRANDMASTER,pNUDUaztjVI1G7Jzoh4aVDMGr_8nDd0sMVeuWOvzwNMC6vB5,MessiOfTFT,394,86,53,4_-cZaXUQM6l7UQGnB69F9007HRO3dyTdKP-7_u0utPCg6YeZCBKMOZvoLFUdNyDa78Y7RBVqBXcGQ
4,GRANDMASTER,VVpwkqbuN3VVwG378diAEHea6jFrq1ADYBQAEstZizthf7k,이렇게 좋은 날,321,94,51,rx1nmyxEddDyQujL2nvjfiHV_LDcBEo-zyf16l3BFnXQExOMoB6tNss-WGlc_GRgcac2tg1GhDgBhg
...,...,...,...,...,...,...,...
68,MASTER,RtwaM2fHJQGrhrEJO6JMtopi4jEGZWZ4M4JB_mNYtes_rlg,iliillliillii1il,81,71,37,8upXuKqJvCQvEmuMiyeHDtXJ7Qkj3Gl4hwFhqorDSVVUs2AL7hcLoqtdTgraRGxtJ8QSnBZQx1qubw
69,MASTER,xEkdkCDDENsw10ck-yZG9ZRvk39WQEoTiqtPrj3w0tS4NYc,luxfan,14,54,27,8pnnl_8NErvSxCy426349zUlZaD3zZIAhwauI8IZugdmuCf0wWvlgBhIHrVhOZBDG6XG7ShdyQPS5g
70,MASTER,R99sYTl70oL4j6iBxmdIU0eCRo-HTgOuAhsetCQ5R8dzcXsjWCb3YqT6-w,COPY C,100,51,22,sV7rWaPHoFjEfvl08t9S7g6GXaExIb_mFB5zJnjxrnpyz194sQ9cPmMXEmDMKxih5iWucZRngigxPA
71,MASTER,z8e2L7AunXVJgA6TwU7w_sjCSX1TyPU2wIavCHkuhvnhbRkpEoV_SynmaA,,21,75,51,None


##### 결측치 제거 - 합친 데이터 테이블에서 puuid 가 null 인 행 제거

In [18]:
combined_df = combined_df.dropna(subset=['puuid'])

combined_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 69 entries, 0 to 72
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   tier          69 non-null     object
 1   summonerId    69 non-null     object
 2   summonerName  69 non-null     object
 3   leaguePoints  69 non-null     int64 
 4   wins          69 non-null     int64 
 5   losses        69 non-null     int64 
 6   puuid         69 non-null     object
dtypes: int64(3), object(4)
memory usage: 4.3+ KB


##### 전처리 완료한 테이블 csv 로 저장

In [19]:
combined_df.to_csv('ranker_name_puuid.csv', index=False)

In [20]:
# 저장된 테이블 확인
pd.read_csv('C:/Users/user/Documents/GitHub/Final-Project-A09-/ranker_name_puuid.csv')

,tier,summonerId,summonerName,leaguePoints,wins,losses,puuid
0,GRANDMASTER,9sxIws6CI6fmK2sw2cAo_Xe8E82DNcR9XtHkWKC6W14SK2w,틈 빈,265,51,10,x5Aluxt-SEq_IUikKvH83sk0EkLXS7CP41bf66M_mBNdsiIUKYdavmHUPDKeWUBL6Ki5WwihBw2kQg
1,GRANDMASTER,BhJKOIvlHJRa_Ra0uNNDlrqN46i6snagNkH_p8VZR28GJA,Teukhan,327,84,55,Qd68F2UMKe7FMb-qpbUs3njG6k_4ZLwr2FRG0zHruSDr9XTtZOZ1cdAXyLd_kGAt0_IheL5QMQLdvQ
2,GRANDMASTER,80HtOC_fs-G1sNBcJFSlZXfNcYfz6he1pHYfBpJHkBs_2TM,COPY O,449,81,38,t_Wf-ad5Drr5FzTmxmkRnsEtBNUf-7PRoTll2k8kxN23sYTKJs2VOrOjeQ418PF31x2QQ2X02RrAtw
3,GRANDMASTER,pNUDUaztjVI1G7Jzoh4aVDMGr_8nDd0sMVeuWOvzwNMC6vB5,MessiOfTFT,394,86,53,4_-cZaXUQM6l7UQGnB69F9007HRO3dyTdKP-7_u0utPCg6YeZCBKMOZvoLFUdNyDa78Y7RBVqBXcGQ
4,GRANDMASTER,VVpwkqbuN3VVwG378diAEHea6jFrq1ADYBQAEstZizthf7k,이렇게 좋은 날,321,94,51,rx1nmyxEddDyQujL2nvjfiHV_LDcBEo-zyf16l3BFnXQExOMoB6tNss-WGlc_GRgcac2tg1GhDgBhg
...,...,...,...,...,...,...,...
64,MASTER,lEu2INdCTm9ritXBYnm72p_4hDkmckM1iB8fT-ht6h2s4xo,liveel,96,89,64,8UxQrTYjRv8TqA7afQmDtq_hDGMprqthuWzOPGcWYmCcsPP6BIH9_G7mZVxgQDvIZ_XIaOrfn_6yxw
65,MASTER,RtwaM2fHJQGrhrEJO6JMtopi4jEGZWZ4M4JB_mNYtes_rlg,iliillliillii1il,81,71,37,8upXuKqJvCQvEmuMiyeHDtXJ7Qkj3Gl4hwFhqorDSVVUs2AL7hcLoqtdTgraRGxtJ8QSnBZQx1qubw
66,MASTER,xEkdkCDDENsw10ck-yZG9ZRvk39WQEoTiqtPrj3w0tS4NYc,luxfan,14,54,27,8pnnl_8NErvSxCy426349zUlZaD3zZIAhwauI8IZugdmuCf0wWvlgBhIHrVhOZBDG6XG7ShdyQPS5g
67,MASTER,R99sYTl70oL4j6iBxmdIU0eCRo-HTgOuAhsetCQ5R8dzcXsjWCb3YqT6-w,COPY C,100,51,22,sV7rWaPHoFjEfvl08t9S7g6GXaExIb_mFB5zJnjxrnpyz194sQ9cPmMXEmDMKxih5iWucZRngigxPA
